In [1]:
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
ANTHROPIC_MODEL ="claude-3-haiku-20240307"
# Store the API key in a variable.
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# Basic Query

In [3]:
# Initialize the model.
llm = ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY, model_name=ANTHROPIC_MODEL, temperature=0.3)

# Define a query as a string.
query = "Give me two dinners to make this week, one has to have chicken and the other salmon."

# Pass the query to the invoke method and print the result.
result = llm.invoke(query)
print(result.content)

Here are two dinner ideas for you this week, one with chicken and one with salmon:

Chicken Dinner:
Baked Lemon Chicken with Roasted Vegetables
- Boneless, skinless chicken breasts or thighs
- Lemon juice, zest, garlic, herbs
- Assorted roasted vegetables (e.g. potatoes, carrots, broccoli, Brussels sprouts)

Salmon Dinner:
Grilled Salmon with Asparagus and Quinoa
- Fresh salmon fillets
- Asparagus spears
- Cooked quinoa
- Lemon, olive oil, salt and pepper

Both of these dinners are healthy, easy to prepare, and feature lean proteins like chicken and salmon. The chicken dish has a bright lemon flavor, while the salmon is paired with nutrient-rich asparagus and quinoa. Let me know if you need any recipe details for these!


In [4]:
# Define a function to use two ingredients as inputs for the query sent to our LLM.
def dinner_recipe(food1, food2):
    # Initialize the model.
    llm = ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY, model_name=ANTHROPIC_MODEL, temperature=0.3)
    # Construct the query using the parameters.
    query = f"Give me two dinners to make this week, one has to have {food1} and the other {food2}."
    # Pass the query to the invoke method which will make a prediction based on the query, and print the result.
    result = llm.invoke(query)
    print(result.content)


# Ask the user for a main ingredient for each day.
food1 = input("What main ingredient would you like in the first day's meal?")
food2 = input("What main ingredient would you like in the second day's meal?")

# Call the function with the two ingredients.
dinner_recipe(food1, food2)

Here are two dinner ideas for you this week:

1. Lemon Chicken Piccata:
   - Ingredients: Chicken breasts, lemon juice, capers, butter, garlic, parsley, and white wine.
   - Preparation: Sauté the chicken in a skillet, then make a lemon-caper sauce to pour over the chicken.

2. Shepherd's Pie:
   - Ingredients: Ground beef or lamb, onions, carrots, peas, mashed potatoes, and cheese.
   - Preparation: Brown the meat, sauté the vegetables, then layer the meat mixture in a baking dish and top with mashed potatoes. Bake until the potatoes are golden brown.


# Chains

In [5]:
# Additional imports for specific chain we'll use.
from langchain.chains import LLMMathChain

In [7]:
# Initialize the model.
llm = ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY, model_name=ANTHROPIC_MODEL, temperature=0)

# Create a math chain for performing calculations.
chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# Set the input query as a text description of a math problem.
query = {"question": "What is the sum of 18 and the cubed root of 24?"}

# Run the chain using the query as input and print the result.
result = chain.invoke(query)
print(result["answer"])



> Entering new LLMMathChain chain...
What is the sum of 18 and the cubed root of 24?Question: What is the sum of 18 and the cubed root of 24?
```text
18 + 24**(1/3)
```
...numexpr.evaluate("18 + 24**(1/3)")...


ValueError: unknown format from LLM: Question: What is the sum of 18 and the cubed root of 24?
```text
18 + 24**(1/3)
```
...numexpr.evaluate("18 + 24**(1/3)")...

# Combining Chains

In [8]:
# Additional imports for specific chains we'll use.
from langchain.chains import LLMChain
from langchain.chains import LLMMathChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate

In [9]:
# Initialize the LLM for text.
chat_llm=ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY, model_name=ANTHROPIC_MODEL, temperature=0.30)

# Initialize the LLM for math.
math_llm=ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY, model_name=ANTHROPIC_MODEL, temperature=0)

# Create a chat chain for creating text.
simple_prompt = ChatPromptTemplate.from_template("{query}")
chat_chain = LLMChain(llm=chat_llm, prompt=simple_prompt)

# Create a math chain for performing calculations.
math_chain = LLMMathChain.from_llm(llm=math_llm)

# Construct the simple sequential chain from the two other chains.
chain = SimpleSequentialChain(
    chains=[chat_chain, math_chain],
    verbose=True
)

# Set the input query for the first chain in the sequence.
query = {"input": "Please write a simple math word problem requiring multiplication."}

# Run the sequential chain using the query as the first input. Print the result.
result = chain.invoke(query)
print(result["output"])



> Entering new SimpleSequentialChain chain...
Here is a simple math word problem requiring multiplication:

Sarah has 4 bags of marbles. Each bag contains 12 marbles. How many marbles does Sarah have in total?

To solve this problem, we need to multiply the number of bags (4) by the number of marbles in each bag (12).

4 bags x 12 marbles per bag = 48 marbles in total.


ValueError: unknown format from LLM: Here is the translation of the math problem into a Python expression using the numexpr library:

Question: Sarah has 4 bags of marbles. Each bag contains 12 marbles. How many marbles does Sarah have in total?
```text
4 * 12
```
...numexpr.evaluate("4 * 12")...

In [10]:
# Additional imports for specific chains we'll use.
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

In [11]:
# Initialize the model.
llm=llm = ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY, model_name=ANTHROPIC_MODEL, temperature=0.3)


# Create a chat chain for creating text.
chat_chain = LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template("{query}"))

# Create a principle for our constitutional chain.
principle = ConstitutionalPrinciple(
    name="Fear of Dogs",
    critique_request="The model should not include dogs of any breed in stories it writes.", # Check the animals you use for their breed and species. Dogs include any breed of dog.Do not include a dog in the story at all. If there's any kind of dog in the story, I would just die.",
    revision_request="Modify the story to be about any animals other than dogs." # Replace any occurrences of a dog of any breed with another animal, like an iguana."
)

# Create a constitutional chain for ensuring the story does not include dogs.
constitutional_chain = ConstitutionalChain.from_llm(
    chain=chat_chain,
    constitutional_principles=[principle],
    llm=llm,
    verbose=True
)

# Set the input query for the chat chain.
query = {"query": "Please give me the main events of a story about three household pets."}

# Run the constitutional chain using the query as the first input.
result = constitutional_chain.invoke(query)
print(result["output"])



> Entering new ConstitutionalChain chain...
Initial response: Here is a possible outline for a story about three household pets:

Main Events:

1. Introduction to the three pets - a dog, a cat, and a hamster - and their different personalities and quirks.

2. The pets get left alone in the house when their owners go out for the day. This sets up the main conflict and adventure.

3. The dog, cat, and hamster get bored and decide to explore the house on their own. This leads to a series of comedic mishaps and misadventures as they get into trouble.

4. The pets accidentally make a mess or cause some damage, leading to a crisis they have to work together to resolve before the owners get home.

5. Through working as a team, the three pets learn to appreciate each other's differences and form an unexpected bond.

6. When the owners return, they're surprised to find the house in disarray but are touched to see the pets have become friends.

7. The story ends with the pets happily coexistin